In [1]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.python.ops import rnn_cell, rnn

In [2]:
from __future__ import division

In [8]:
import time

In [3]:
!ls '/home/htan/proj/TensorFlow/data/visibility/'

ASOS_alone.pickle	     CC_Under_sSample.pickle
ASOS+NWP.pickle		     KORD_RUC_RAP_Hourly_20051031-20150301.csv
ASOS+NWP_time_serial.pickle  rough_visibility.pickle
ASOS_time_serial.pickle      visibilityDataNoLagsNewPreds.csv


In [4]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP_time_serial.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    #train_dataset= save['train_dataset']
    #validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    #test_old = save['v_t_dataset']
    train_old = save['t_v_dataset']
    del save

#train_time = train_dataset['time']
#train_data = train_dataset['data']
#train_label = train_dataset['label']
#validate_time = validate_dataset['time']
#validate_data = validate_dataset['data']
#validate_label = validate_dataset['label']
#test_time = test_dataset['time']
test_data = test_dataset['data']
test_label = test_dataset['label']
#test_old_data = test_old['data']
#test_old_label = test_old['label']
train_old_data = train_old['data']
train_old_label = train_old['label']

In [5]:
print(test_data.shape, test_label.shape)
print(train_old_data.shape, train_old_label.shape)

(17664, 8, 82) (17664, 8, 1)
(70656, 8, 82) (70656, 8, 1)


In [6]:
#dataset normalize
mean = train_old_data.mean(axis = (0,1))
std = train_old_data.std(axis = (0,1))
print(mean.shape, std.shape)
train_data_n = (train_old_data - mean)/std
#validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std

(82,) (82,)


In [7]:
train_label = train_old_label

# Simple LSTM regression Model 

In [9]:
a = test_label[:10, 7, :].shape


In [13]:
n_features = 82
n_steps = 8
n_labels = 1

n_hidden = 140
total_size = train_old_label.shape[0]

In [19]:


graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder("float32", [None, n_steps, n_features])
    labels = tf.placeholder("float32", [None, n_labels])
    
    weights = {
        'hidden': tf.Variable(tf.random_normal([n_features, n_hidden])),
        'out': tf.Variable(tf.random_normal([n_hidden, n_labels]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels]))
    }
    
    def RNN(x, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features])
        
        # Linear activation
        x = tf.matmul(x, w['hidden']) + b['hidden']
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        x = tf.split(0, n_steps, x)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs, weights, biases)
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.abs(pred - labels))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.15
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 250, 0.90, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    MAE = tf.reduce_mean(tf.abs(pred - labels))

In [20]:
st = time.time()
batch_size = 128
steps = 10000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs:test_data_n, labels:test_label[:,-1,:]}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n[off:off+batch_size, :, :]
        batch_label = train_label[off:off+batch_size, -1, :]
        feed_dict = {inputs:batch_data, labels:batch_label}
        l, _, r = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 100 == 0:
            test_mae = MAE.eval(feed_dict=test_feed_dict)
            print('step: %d, LR = %f, min batch loss = %f, test MAE = %f' % (step, r, l, test_mae))
            #print('batch_data = %d %d %d, batch_label = %d, predicate = %f' % (batch_data[0, 0, 0],batch_data[0, 1, 0],batch_data[0, 2, 0], batch_label[0, 0], p))
et = time.time()
print(et - st)

Initialized
step: 0, LR = 0.150000, min batch loss = 8.971102, test MAE = 8.520988
step: 100, LR = 0.150000, min batch loss = 1.961214, test MAE = 2.389309
step: 200, LR = 0.150000, min batch loss = 1.589390, test MAE = 1.402911
step: 300, LR = 0.135000, min batch loss = 2.388237, test MAE = 1.290621
step: 400, LR = 0.135000, min batch loss = 1.566636, test MAE = 1.133223
step: 500, LR = 0.121500, min batch loss = 0.622787, test MAE = 1.012790
step: 600, LR = 0.121500, min batch loss = 0.896475, test MAE = 0.987309
step: 700, LR = 0.121500, min batch loss = 1.071309, test MAE = 0.971154
step: 800, LR = 0.109350, min batch loss = 0.889713, test MAE = 0.867428
step: 900, LR = 0.109350, min batch loss = 1.921682, test MAE = 0.944717
step: 1000, LR = 0.098415, min batch loss = 0.384690, test MAE = 0.812613
step: 1100, LR = 0.098415, min batch loss = 0.416397, test MAE = 0.804676
step: 1200, LR = 0.098415, min batch loss = 0.737781, test MAE = 0.831845
step: 1300, LR = 0.088573, min batch l

# Simple LSTM classification Model

In [21]:
label1 = train_label.reshape(train_label.shape[0], train_label.shape[1])
label2 = test_label.reshape(test_label.shape[0], test_label.shape[1])

In [22]:
#Do classification later
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(label1)
b_test_label = pre.transform(label2)

In [23]:
c_train_label = 1 - b_train_label
r, _ = (c_train_label == 1).nonzero()
r.shape, c_train_label.shape, c_train_label.reshape(c_train_label.shape[0]*c_train_label.shape[1], 1).shape

((10810,), (70656, 8), (565248, 1))

In [24]:
c_train_label = 1 - b_train_label
c_test_label = 1 - b_test_label

In [25]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [26]:
v_train_label_t = makeIndicatorVars(c_train_label.reshape(c_train_label.shape[0]*c_train_label.shape[1], 1))
v_test_label_t = makeIndicatorVars(c_test_label.reshape(c_test_label.shape[0]*c_test_label.shape[1], 1))

In [27]:
v_train_label = v_train_label_t.reshape(c_train_label.shape[0], c_train_label.shape[1], 2)
v_test_label = v_test_label_t.reshape(c_test_label.shape[0], c_test_label.shape[1], 2)


In [28]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE
sm = SMOTE(ratio = 0.1, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, v_train_label)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [29]:
#shuffle the data set
arr = np.arange(v_train_label.shape[0])
np.random.shuffle(arr)
train_data_n_s =  train_data_n[arr]
train_label_v_s = v_train_label[arr]

Simple LSTM Model

In [57]:
n_features = 82
n_steps = 8
n_labels = 2
n_hidden = 140
total_size = v_train_label.shape[0]
ratio = 0.27

In [58]:


graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder("float32", [None, n_steps, n_features])
    labels = tf.placeholder("float32", [None, n_labels])
    
    weights = {
        'hidden': tf.Variable(tf.random_normal([n_features, n_hidden])),
        'out': tf.Variable(tf.random_normal([n_hidden, n_labels]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels]))
    }
    
    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN) 
        FP_percent = FP / (FP + TN) 
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f_score = 2 * precision * recall / (precision + recall + 1)
        return precision, recall, f_score
    
    def RNN(x, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features])
        
        # Linear activation
        x = tf.matmul(x, w['hidden']) + b['hidden']
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        x = tf.split(0, n_steps, x)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs, weights, biases)
    class_weight = tf.constant([ratio, 1.0 - ratio])
    weighted_pred = tf.mul(pred, class_weight) # shape [batch_size, 2]
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(weighted_pred, labels)) # Softmax loss
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.15
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90 , staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_acc, FP, FN, TP, TN = acc(pred, labels)
    pre, rec, f_s = PRC(FP, FN, TP, TN)

In [59]:
batch_size = 128
steps = 10000
st = time.time()
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs:test_data_n, labels:v_test_label[:,-1,:]}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n_s[off:off+batch_size, :, :]
        batch_label = train_label_v_s[off:off+batch_size, -1, :]
        feed_dict = {inputs:batch_data, labels:batch_label}
        l, _, r = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 100 == 0:
            tp, fp, fn, tn, precision, recall, f_score= session.run([TP, FP, FN, TN, pre, rec, f_s], feed_dict=test_feed_dict)
            print("TP = %d, FP = %d, FN = %d, TN = %d" % (tp, fp, fn, tn))
            print("precision = %f, recall = %f, f_score = %f" % (precision, recall, f_score))
            #print('batch_data = %d %d %d, batch_label = %d, predicate = %f' % (batch_data[0, 0, 0],batch_data[0, 1, 0],batch_data[0, 2, 0], batch_label[0, 0], p))
            
et = time.time()
print(et-st)

Initialized
TP = 4, FP = 247, FN = 310, TN = 17103
precision = 0.015936, recall = 0.012739, f_score = 0.000395
TP = 61, FP = 66, FN = 253, TN = 17284
precision = 0.480315, recall = 0.194268, f_score = 0.111442
TP = 70, FP = 112, FN = 244, TN = 17238
precision = 0.384615, recall = 0.222930, f_score = 0.106675
TP = 192, FP = 422, FN = 122, TN = 16928
precision = 0.312704, recall = 0.611465, f_score = 0.198743
TP = 198, FP = 679, FN = 116, TN = 16671
precision = 0.225770, recall = 0.630573, f_score = 0.153381
TP = 169, FP = 386, FN = 145, TN = 16964
precision = 0.304505, recall = 0.538217, f_score = 0.177878
TP = 154, FP = 391, FN = 160, TN = 16959
precision = 0.282569, recall = 0.490446, f_score = 0.156327
TP = 205, FP = 587, FN = 109, TN = 16763
precision = 0.258838, recall = 0.652866, f_score = 0.176792
TP = 146, FP = 241, FN = 168, TN = 17109
precision = 0.377261, recall = 0.464968, f_score = 0.190437
TP = 219, FP = 649, FN = 95, TN = 16701
precision = 0.252304, recall = 0.697452, f_s